In [410]:
import numpy as np
import pandas as pd

from collections import Counter

In [411]:
def construir_arbol(instancias, etiquetas):
    print("Instancias")
    print(instancias)
    # ALGORITMO RECURSIVO para construcción de un árbol de decisión binario. 
    # Suponemos que estamos parados en la raiz del árbol y tenemos que decidir cómo construirlo. 
    ganancia, pregunta = encontrar_mejor_atributo_y_corte(instancias, etiquetas)
    # Criterio de corte: ¿Hay ganancia?
    if ganancia == 0:
        #  Si no hay ganancia en separar, no separamos. 
        return Hoja(etiquetas)
    else: 
        # Si hay ganancia en partir el conjunto en 2
        instancias_cumplen, etiquetas_cumplen, instancias_no_cumplen, etiquetas_no_cumplen = partir_segun(pregunta, instancias, etiquetas)
        # partir devuelve instancias y etiquetas que caen en cada rama (izquierda y derecha)

        # Paso recursivo (consultar con el computador más cercano)
        sub_arbol_izquierdo = construir_arbol(instancias_cumplen, etiquetas_cumplen)
        sub_arbol_derecho   = construir_arbol(instancias_no_cumplen, etiquetas_no_cumplen)
        # los pasos anteriores crean todo lo que necesitemos de sub-árbol izquierdo y sub-árbol derecho
        
        # sólo falta conectarlos con un nodo de decisión:
        return Nodo_De_Decision(pregunta, sub_arbol_izquierdo, sub_arbol_derecho)

In [412]:
# Definición de la estructura del árbol. 

class Hoja:
    #  Contiene las cuentas para cada clase (en forma de diccionario)
    #  Por ejemplo, {'Si': 2, 'No': 2}
    def __init__(self, etiquetas):
        self.cuentas = dict(Counter(etiquetas))


class Nodo_De_Decision:
    # Un Nodo de Decisión contiene preguntas y una referencia al sub-árbol izquierdo y al sub-árbol derecho
     
    def __init__(self, pregunta, sub_arbol_izquierdo, sub_arbol_derecho):
        self.pregunta = pregunta
        self.sub_arbol_izquierdo = sub_arbol_izquierdo
        self.sub_arbol_derecho = sub_arbol_derecho
        
        
# Definición de la clase "Pregunta"
class Pregunta:
    def __init__(self, atributo, valor):
        self.atributo = atributo
        self.valor = valor
    
    def cumple(self, instancia):
        # Devuelve verdadero si la instancia cumple con la pregunta
        return instancia[self.atributo] == self.valor
    
    def __repr__(self):
        return "¿Es el valor para {} igual a {}?".format(self.atributo, self.valor)

In [413]:
def gini(etiquetas):
    #print("Etiquetas para calculo de gini: ", etiquetas)
    diccionario = dict(Counter(etiquetas))
    suma = 0
    for etiqueta in diccionario.keys():
        suma += (diccionario[etiqueta]/len(etiquetas))**2
        #print(etiqueta, diccionario[etiqueta])
    impureza = 1 - suma
    return impureza

def ganancia_gini(instancias, etiquetas_rama_izquierda, etiquetas_rama_derecha):
    etiquetas = np.concatenate((etiquetas_rama_izquierda,etiquetas_rama_derecha))
    n_izq = len(etiquetas_rama_izquierda)
    n_der = len(etiquetas_rama_derecha)
    n = len(etiquetas)
    
    gini_total = gini(etiquetas)
    gini_izq = gini(etiquetas_rama_izquierda)
    gini_der = gini(etiquetas_rama_derecha)
    
    ganancia_gini = gini_total - ((n_izq/n)*gini_izq + (n_der/n)*gini_der)
    
    return ganancia_gini


def partir_segun(pregunta, instancias, etiquetas):
    # Esta función debe separar instancias y etiquetas según si cada instancia cumple o no con la pregunta (ver método 'cumple')
    # COMPLETAR (recomendamos utilizar máscaras para este punto)
    instancias_cumplen = pd.DataFrame(columns=instancias.columns)
    instancias_no_cumplen = pd.DataFrame(columns=instancias.columns)
     
    columna_etiqueta = pd.DataFrame(data=etiquetas, index=instancias.index, columns=['etiqueta'])
    instancias_con_etiqueta = pd.concat([instancias, columna_etiqueta], axis=1)
    instancias_cumplen_etiqueta = instancias_con_etiqueta.where(pregunta.cumple(instancias_con_etiqueta)).dropna()
    instancias_no_cumplen_etiqueta = instancias_con_etiqueta.mask(pregunta.cumple(instancias_con_etiqueta)).dropna()
    etiquetas_cumplen = instancias_cumplen_etiqueta.loc[:,'etiqueta'].tolist()
    etiquetas_no_cumplen = instancias_no_cumplen_etiqueta.loc[:,'etiqueta'].tolist()
    

    #print("Instancias que cumplen {}".format(pregunta))
    #print(instancias_cumplen_etiqueta)
    #print("Instancias que no cumplen {}".format(pregunta))
    #print(instancias_no_cumplen_etiqueta)
    
    del instancias_cumplen_etiqueta['etiqueta']
    del instancias_no_cumplen_etiqueta['etiqueta']
    
    instancias_cumplen = instancias_cumplen_etiqueta
    instancias_no_cumplen = instancias_no_cumplen_etiqueta
    
    return instancias_cumplen, etiquetas_cumplen, instancias_no_cumplen, etiquetas_no_cumplen

In [414]:
def encontrar_mejor_atributo_y_corte(instancias, etiquetas):
    max_ganancia = 0
    mejor_pregunta = None
    for columna in instancias.columns:
        print("Columna: " + columna)
        for valor in set(instancias[columna]):
            print("Valor: " + valor)
            # Probando corte para atributo y valor
            pregunta = Pregunta(columna, valor)
            _, etiquetas_rama_izquierda, _, etiquetas_rama_derecha = partir_segun(pregunta, instancias, etiquetas)
   
            ganancia = ganancia_gini(instancias, etiquetas_rama_izquierda, etiquetas_rama_derecha)
            print("La ganancia para la pregunta {}, es {}".format(pregunta, ganancia))
            if ganancia > max_ganancia:
                max_ganancia = ganancia
                mejor_pregunta = pregunta
    print("La mejor pregunta es {}, con una ganancia de {}".format(mejor_pregunta, max_ganancia))        
    return max_ganancia, mejor_pregunta


def imprimir_arbol(arbol, spacing=""):
    if isinstance(arbol, Hoja):
        print (spacing + "Hoja:", arbol.cuentas)
        return

    print (spacing + str(arbol.pregunta))

    print (spacing + '--> True:')
    imprimir_arbol(arbol.sub_arbol_izquierdo, spacing + "  ")

    print (spacing + '--> False:')
    imprimir_arbol(arbol.sub_arbol_derecho, spacing + "  ")

In [415]:
X = pd.DataFrame([["Sol","Calor","Alta","Debil"],
                ["Sol","Calor","Alta","Fuerte"],
                ["Nublado","Calor","Alta","Debil"],
                ["Lluvia","Templado","Alta","Debil"],
                ["Lluvia","Frio","Normal","Debil"],
                ["Lluvia","Frio","Normal","Fuerte"],
                ["Nublado","Frio","Normal","Fuerte"],
                ["Sol","Templado","Alta","Debil"],
                ["Sol","Frio","Normal","Debil"],
                ["Lluvia","Templado","Normal","Debil"],
                ["Sol","Templado","Normal","Fuerte"],
                ["Nublado","Templado","Alta","Fuerte"],
                ["Nublado","Calor","Normal","Debil"],
                ["Lluvia","Templado","Alta","Fuerte"]],
                columns = ['Cielo', 'Temperatura', 'Humedad', 'Viento'])

y = ['No', 'No', 'Si', 'Si', 'Si', 'No', 'Si', 'No', 'Si', 'Si', 'Si', 'Si', 'Si', 'No']

display(X)
display(y)

,Cielo,Temperatura,Humedad,Viento
0,Sol,Calor,Alta,Debil
1,Sol,Calor,Alta,Fuerte
2,Nublado,Calor,Alta,Debil
3,Lluvia,Templado,Alta,Debil
4,Lluvia,Frio,Normal,Debil
5,Lluvia,Frio,Normal,Fuerte
6,Nublado,Frio,Normal,Fuerte
7,Sol,Templado,Alta,Debil
8,Sol,Frio,Normal,Debil
9,Lluvia,Templado,Normal,Debil


['No',
 'No',
 'Si',
 'Si',
 'Si',
 'No',
 'Si',
 'No',
 'Si',
 'Si',
 'Si',
 'Si',
 'Si',
 'No']

In [416]:
arbol = construir_arbol(X, y)
imprimir_arbol(arbol)

Instancias
      Cielo Temperatura Humedad  Viento
0       Sol       Calor    Alta   Debil
1       Sol       Calor    Alta  Fuerte
2   Nublado       Calor    Alta   Debil
3    Lluvia    Templado    Alta   Debil
4    Lluvia        Frio  Normal   Debil
5    Lluvia        Frio  Normal  Fuerte
6   Nublado        Frio  Normal  Fuerte
7       Sol    Templado    Alta   Debil
8       Sol        Frio  Normal   Debil
9    Lluvia    Templado  Normal   Debil
10      Sol    Templado  Normal  Fuerte
11  Nublado    Templado    Alta  Fuerte
12  Nublado       Calor  Normal   Debil
13   Lluvia    Templado    Alta  Fuerte
Columna: Cielo
Valor: Sol
La ganancia para la pregunta ¿Es el valor para Cielo igual a Sol?, es 0.06553287981859401
Valor: Lluvia
La ganancia para la pregunta ¿Es el valor para Cielo igual a Lluvia?, es 0.002040816326530581
Valor: Nublado
La ganancia para la pregunta ¿Es el valor para Cielo igual a Nublado?, es 0.10204081632653056
Columna: Temperatura
Valor: Frio
La ganancia para la pre

     Cielo Temperatura Humedad  Viento
3   Lluvia    Templado    Alta   Debil
13  Lluvia    Templado    Alta  Fuerte
Columna: Cielo
Valor: Lluvia
La ganancia para la pregunta ¿Es el valor para Cielo igual a Lluvia?, es 0.0
Columna: Temperatura
Valor: Templado
La ganancia para la pregunta ¿Es el valor para Temperatura igual a Templado?, es 0.0
Columna: Humedad
Valor: Alta
La ganancia para la pregunta ¿Es el valor para Humedad igual a Alta?, es 0.0
Columna: Viento
Valor: Fuerte
La ganancia para la pregunta ¿Es el valor para Viento igual a Fuerte?, es 0.5
Valor: Debil
La ganancia para la pregunta ¿Es el valor para Viento igual a Debil?, es 0.5
La mejor pregunta es ¿Es el valor para Viento igual a Fuerte?, con una ganancia de 0.5
Instancias
     Cielo Temperatura Humedad  Viento
13  Lluvia    Templado    Alta  Fuerte
Columna: Cielo
Valor: Lluvia
La ganancia para la pregunta ¿Es el valor para Cielo igual a Lluvia?, es 0.0
Columna: Temperatura
Valor: Templado
La ganancia para la pregunta ¿Es

## Resultado esperado

```
¿Es el valor para Cielo igual a Nublado?
--> True:
  ¿Es el valor para Temperatura igual a Frio?
  --> True:
    Hoja: {'Si': 1}
  --> False:
    ¿Es el valor para Temperatura igual a Templado?
    --> True:
      Hoja: {'Si': 1}
    --> False:
      Hoja: {'Si': 2}
--> False:
  ¿Es el valor para Humedad igual a Normal?
  --> True:
    ¿Es el valor para Viento igual a Fuerte?
    --> True:
      Hoja: {'No': 1, 'Si': 1}
    --> False:
      ¿Es el valor para Cielo igual a Sol?
      --> True:
        Hoja: {'Si': 1}
      --> False:
        Hoja: {'Si': 2}
  --> False:
    ¿Es el valor para Cielo igual a Sol?
    --> True:
      ¿Es el valor para Temperatura igual a Templado?
      --> True:
        Hoja: {'No': 1}
      --> False:
        Hoja: {'No': 2}
    --> False:
      Hoja: {'Si': 1, 'No': 1}
```

## Parte 2 (opcional)
Protocolo sklearn para clasificadores. Completar el protocolo requerido por sklearn. Deben completar la función predict utilizando el árbol para predecir valores de nuevas instancias. 


In [417]:
def predecir(arbol, x_t):
    if isinstance(arbol, Hoja):
        return max(arbol.cuentas, key=arbol.cuentas.get)
    
    if(arbol.pregunta.cumple(x_t)):
        return predecir(arbol.sub_arbol_izquierdo, x_t)
    else:
        return predecir(arbol.sub_arbol_derecho, x_t)
        
class MiClasificadorArbol(): 
    def __init__(self):
        self.arbol = None
        self.columnas = ['Cielo', 'Temperatura', 'Humedad', 'Viento']
    
    def fit(self, X_train, y_train):
        self.arbol = construir_arbol(pd.DataFrame(X_train, columns=self.columnas), y_train)
        return self
    
    def predict(self, X_test):
        predictions = []
        for x_t in X_test:
            x_t_df = pd.DataFrame([x_t], columns=self.columnas).iloc[0]
            prediction = predecir(self.arbol, x_t_df) 
            print(x_t, "predicción ->", prediction)
            predictions.append(prediction)
        return predictions
    
    def score(self, X_test, y_test):
        y_pred = self.predict(X_test)
        
        accuracy = sum(y_i == y_j for (y_i, y_j) in zip(y_pred, y_test)) / len(y_test)
        return accuracy
        

# Ejemplo de uso
clf = MiClasificadorArbol()

# Tomar en cuenta que sklearn espera numpy arrays:
clf.fit(np.array(X), y)
clf.score(np.array(X), y)


Instancias
      Cielo Temperatura Humedad  Viento
0       Sol       Calor    Alta   Debil
1       Sol       Calor    Alta  Fuerte
2   Nublado       Calor    Alta   Debil
3    Lluvia    Templado    Alta   Debil
4    Lluvia        Frio  Normal   Debil
5    Lluvia        Frio  Normal  Fuerte
6   Nublado        Frio  Normal  Fuerte
7       Sol    Templado    Alta   Debil
8       Sol        Frio  Normal   Debil
9    Lluvia    Templado  Normal   Debil
10      Sol    Templado  Normal  Fuerte
11  Nublado    Templado    Alta  Fuerte
12  Nublado       Calor  Normal   Debil
13   Lluvia    Templado    Alta  Fuerte
Columna: Cielo
Valor: Sol
La ganancia para la pregunta ¿Es el valor para Cielo igual a Sol?, es 0.06553287981859401
Valor: Lluvia
La ganancia para la pregunta ¿Es el valor para Cielo igual a Lluvia?, es 0.002040816326530581
Valor: Nublado
La ganancia para la pregunta ¿Es el valor para Cielo igual a Nublado?, es 0.10204081632653056
Columna: Temperatura
Valor: Frio
La ganancia para la pre

     Cielo Temperatura Humedad  Viento
0      Sol       Calor    Alta   Debil
1      Sol       Calor    Alta  Fuerte
3   Lluvia    Templado    Alta   Debil
7      Sol    Templado    Alta   Debil
13  Lluvia    Templado    Alta  Fuerte
Columna: Cielo
Valor: Sol
La ganancia para la pregunta ¿Es el valor para Cielo igual a Sol?, es 0.11999999999999983
Valor: Lluvia
La ganancia para la pregunta ¿Es el valor para Cielo igual a Lluvia?, es 0.11999999999999983
Columna: Temperatura
Valor: Calor
La ganancia para la pregunta ¿Es el valor para Temperatura igual a Calor?, es 0.05333333333333318
Valor: Templado
La ganancia para la pregunta ¿Es el valor para Temperatura igual a Templado?, es 0.05333333333333318
Columna: Humedad
Valor: Alta
La ganancia para la pregunta ¿Es el valor para Humedad igual a Alta?, es 0.0
Columna: Viento
Valor: Fuerte
La ganancia para la pregunta ¿Es el valor para Viento igual a Fuerte?, es 0.05333333333333318
Valor: Debil
La ganancia para la pregunta ¿Es el valor para Vien

1.0